In [1]:
# Загрузим необходимые библиотеки
import pandas as pd
from tqdm import tqdm
from src.category_tree.category_tree import CategoryTree
import requests
import numpy as np

In [ ]:
# Зададим константы
RANDOM_STATE = 42

In [ ]:
# Загрузим иерархию категорий
category_tree = CategoryTree(category_tree_path='data/raw/category_tree.csv')
categor = pd.read_csv('data/raw/category_tree.csv')

# Загрузим ошибочно размеченные примеры
df = pd.read_parquet('data/processed/bad_labeled.parquet', columns=['source_name', 'cat_id', 'hash_id'])

# Отфильтруем только те примеры, которые относятся к категориям на уровне перед листом в дереве
pre_leaf_nodes = set(categor[categor.cat_id.isin(category_tree.leaf_nodes)].parent_id.to_list())
df = df[df.cat_id.isin(pre_leaf_nodes)]

# Перемешаем отфильтрованные данные случайным образом, чтобы избежать последовательных закономерностей
df = df.sample(frac=1,random_state=RANDOM_STATE).reset_index(drop=True)

# Добавим текстовые названия категорий к данным
df = df.merge(categor, on='cat_id', how='left')

In [4]:
# Пытаемся загрузить ранее сохраненный прогресс обработки данных
try:
    df_save = pd.read_csv('data/processed/bad_labeled_qwen2.5:3b.csv')
except FileNotFoundError:
    # Если файл не найден
    df_save = pd.DataFrame(columns=['hash_id', 'source_name', 'pred_cat_id'])
df_save.shape

(4181, 3)

In [5]:
# Настройки Ollama
OLLAMA_URL = 'http://127.0.0.1:13537/api/chat'
MODEL_NAME = 'qwen2.5:3b'

def ask_ollama(prompt, history=[]):
    
    try:
        response = requests.post(OLLAMA_URL, json={
            'model': MODEL_NAME,
            'messages': history + [{'role': 'user', 'content': prompt}],
            'temperature': 0.2,
            'stream': False
        })
        response.raise_for_status()
        return response.json()['message']['content'].strip()
    except Exception as e:
        print(f"Error requesting Ollama: {e}")
        return None

# Подготовим данные
start_categor = categor[categor['parent_id'].isna()]['cat_name'].tolist()
categor_dict = categor.groupby('parent_id')['cat_name'].apply(list).to_dict()

# Вытаскиваем нужные колонки из датафрейма df
source_name_list = df['source_name'].tolist()
hash_id_list = df['hash_id'].tolist()
cat_name_list = df['cat_name'].tolist()
cat_id_list = df['cat_id'].tolist()

# Создадим список для хранения прогнозов
cat_id_pred = []

# Определим начальный индекс для обработки
start_index = df_save.shape[0]
count = 0 

for name, hash_id, cat_name, cat_id in tqdm(
    zip(source_name_list[start_index:], hash_id_list[start_index:], cat_name_list[start_index:], cat_id_list[start_index:]), 
    total=len(source_name_list[start_index:])
):
    count += 1

    # Найдем подкатегорию для текущей категории
    subcategories = categor_dict.get(cat_id, [])

    # Отформатируем список подкатегорий в виде строки
    lst = '\n - ' + '\n - '.join(subcategories)
    cat = cat_name

    # Инициализируем журнал разговоров с помощью системных инструкций
    chat_history = [{
        'role': 'system',
        'content': (
            'Ты полезный помощник по определению категории товара и списка категорий, который строго следует инструкции. '
            'Ты всегда выбираешь только одну категорию из предложенного списка, без лишних слов.'
        )
    }]

    prompt = (
        f"Товар: '{name}'.\n"
        f"Текущая категория: {cat}.\n"
        f"Выбери наиболее подходящую подкатегорию из списка:\n{lst}\n"
        f"Ответ должен быть строго только названием категории из списка, без лишних слов."
    )

    answer = ask_ollama(prompt, chat_history)
    if not answer:
        # Если запрос Ollama завершается неудачей, добавьте результат сбоя и прервем цикл
        cat_id_pred.append([hash_id, name, None])
        break

    # Удалим лишние символы
    answer = answer.strip().strip('.')

    chat_history.append({'role': 'user', 'content': prompt})
    chat_history.append({'role': 'assistant', 'content': answer})

    # Найдем идентификатор подкатегории на основе ответа Олламы
    mask = (categor['cat_name'] == answer) & (categor['parent_id'] == cat_id)
    category = categor.loc[mask]

    if category.empty:
        cat_id_pred.append([hash_id, name, np.nan])
    else:
        cat_id_pred.append([hash_id, name, answer])

    # Сохраняем прогресс каждые 10 шагов
    if count % 10 == 0:
        df_save = pd.concat([df_save, pd.DataFrame(cat_id_pred, columns=['hash_id', 'source_name', 'pred_cat_id'])])
        df_save.to_csv('data/processed/bad_labeled_qwen2.5:3b.csv', index=False, na_rep='NaN')
        cat_id_pred = []

    if count % 1000 == 0:
        print(df_save.shape)

# Сохраним все оставшиеся прогнозы
if cat_id_pred:
    df_save = pd.concat([df_save, pd.DataFrame(cat_id_pred, columns=['hash_id', 'source_name', 'pred_cat_id'])])
    df_save.to_csv('data/processed/bad_labeled_qwen2.5:3b.csv', index=False, na_rep='NaN')

100%|██████████| 19/19 [00:59<00:00,  3.11s/it]
